In [1]:
# ✅ Install required packages
!pip install mne neurokit2 matplotlib tqdm --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.1/693.1 kB 38.4 MB/s eta 0:00:00


In [2]:
!pip install pymatreader h5py mne neurokit2

In [3]:
# ✅ Import libraries
import os
import mne
import neurokit2 as nk
import numpy as np
import matplotlib.pyplot as plt
import pickle
import gc
from tqdm import tqdm

In [4]:
# ✅ Parameters
SAVE_PLOTS = False  # Set to True to save ECG plots
dataset_path = "/content/drive/MyDrive/EntireDataset_240GB"
output_root = "/content/output_ecg_data"
output_images_dir = "/content/output_ecg_images"
os.makedirs(output_root, exist_ok=True)
if SAVE_PLOTS:
    os.makedirs(output_images_dir, exist_ok=True)



In [5]:
num_conditions = 3
num_subconditions = 3
num_trials = 54
max_time = 26000

# ✅ Event dictionary
event_dicts = {
    "five": {
        "justlisten/five": 1,
        "memory/correct/five": 29,
        "memory/incorrect/five": 28
    },
    "nine": {
        "justlisten/nine": 2,
        "memory/correct/nine": 31,
        "memory/incorrect/nine": 30
    },
    "thirteen": {
        "justlisten/thirteen": 3,
        "memory/correct/thirteen": 33,
        "memory/incorrect/thirteen": 32
    }
}

In [6]:
epoch_durations = {"five": 10, "nine": 18, "thirteen": 26}
reject_criteria = dict(ecg=1000000e-6)

excluded_subjects = {
    "sub-013", "sub-014", "sub-015", "sub-016", "sub-017", "sub-018", "sub-019",
    "sub-020", "sub-021", "sub-022", "sub-023", "sub-024", "sub-025", "sub-026",
    "sub-027", "sub-028", "sub-029", "sub-030", "sub-031", "sub-037", "sub-066"
}

# ✅ Locate .set files
def find_set_files(root_dir):
    set_files = []
    for root, _, files in os.walk(root_dir):
        for file in files:
            if file.endswith(".set") and "task-memory_ecg" in file:
                set_files.append(os.path.join(root, file))
    return set_files

set_files = find_set_files(dataset_path)

In [7]:
subject_list = []
data_array = []

# ✅ Loop through all files
for file_path in tqdm(set_files, desc="Processing subjects"):
    subject_id = os.path.basename(file_path).split("_")[0]
    if subject_id in excluded_subjects:
        continue

    try:
        print(f"\n🔍 Processing {subject_id}...")
        raw = mne.io.read_raw_eeglab(file_path, preload=True)
        raw.set_channel_types({"ECG": "ecg"})
        events, _ = mne.events_from_annotations(raw)

        # Preallocate subject 4D array (condition × subcondition × trial × time)
        subject_data = np.full((num_conditions, num_subconditions, num_trials, max_time), np.nan, dtype=np.float32)

        for condition_idx, (condition, event_dict) in enumerate(event_dicts.items()):
            available_event_ids = set(events[:, 2])
            filtered_event_dict = {k: v for k, v in event_dict.items() if v in available_event_ids}
            if not filtered_event_dict:
                print(f"⚠️ Skipping condition '{condition}' for {subject_id}: No matching events.")
                continue

            try:
                epochs = mne.Epochs(
                    raw, events, tmin=-3, tmax=epoch_durations[condition],
                    event_id=filtered_event_dict, preload=True,
                    reject=reject_criteria, baseline=None
                )
            except Exception as e:
                print(f"❌ [{subject_id}] Epoching failed: {e}")
                continue

            for trial_idx in range(min(num_trials, len(epochs))):
                try:
                    selected_epoch = epochs[trial_idx]
                    ecg_signal = selected_epoch.get_data(picks="ECG")[0, 0, :]

                    if np.isnan(ecg_signal).all() or len(ecg_signal) < 1000 or np.std(ecg_signal) < 1e-6:
                        continue

                    ecg_cleaned = nk.ecg_clean(ecg_signal, sampling_rate=1000)
                    _, rpeaks_dict = nk.ecg_peaks(ecg_cleaned, sampling_rate=1000)
                    rpeaks = rpeaks_dict["ECG_R_Peaks"]

                    if len(rpeaks) < 3:
                        continue

                    signals, info = nk.ecg_process(ecg_cleaned, sampling_rate=1000)

                    # Event and subcondition mapping
                    event_code = epochs.events[trial_idx, 2]
                    if event_code in event_dict.values():
                        event_label = list(event_dict.keys())[list(event_dict.values()).index(event_code)]
                        subcondition_idx = 0 if "justlisten" in event_label else (1 if "memory/correct" in event_label else 2)
                    else:
                        continue

                    time_length = len(ecg_signal)
                    if time_length > max_time:
                        ecg_data_trimmed = ecg_signal[:max_time]
                    else:
                        ecg_data_trimmed = np.pad(ecg_signal, (0, max_time - time_length), constant_values=np.nan)

                    subject_data[condition_idx, subcondition_idx, trial_idx, :] = ecg_data_trimmed.astype(np.float32)

                    # Save ECG plot
                    if SAVE_PLOTS:
                        fig = nk.ecg_plot(signals, info)
                        plot_path = os.path.join(output_images_dir, f"{subject_id}_{condition}_{trial_idx}.png")
                        fig.savefig(plot_path)
                        plt.close(fig)

                except Exception as e:
                    print(f"⚠️ [{subject_id}] Trial {trial_idx} processing failed: {e}")
                    continue

        subject_list.append(subject_id)
        data_array.append(subject_data)

        # ✅ Clean up memory
        del raw, epochs, subject_data
        gc.collect()

    except Exception as e:
        print(f"❌ Failed to process {subject_id}: {e}")
        continue

# ✅ Final 5D data array
final_data_array = np.array(data_array, dtype=np.float32)

Processing subjects:   0%|          | 0/83 [00:00<?, ?it/s]


🔍 Processing sub-033...


/usr/local/lib/python3.11/dist-packages/pymatreader/utils.py:168: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn(


Used Annotations descriptions: [np.str_('500105'), np.str_('500109'), np.str_('500113'), np.str_('500205'), np.str_('500209'), np.str_('500213'), np.str_('500305'), np.str_('500309'), np.str_('500313'), np.str_('500405'), np.str_('500409'), np.str_('500413'), np.str_('500505'), np.str_('500509'), np.str_('500513'), np.str_('500609'), np.str_('500613'), np.str_('500709'), np.str_('500713'), np.str_('500809'), np.str_('500813'), np.str_('500909'), np.str_('500913'), np.str_('501013'), np.str_('501113'), np.str_('501213'), np.str_('501313'), np.str_('6001050'), np.str_('6001051'), np.str_('6001090'), np.str_('6001091'), np.str_('6001130'), np.str_('6001131'), np.str_('6002050'), np.str_('6002051'), np.str_('6002090'), np.str_('6002091'), np.str_('6002130'), np.str_('6002131'), np.str_('6003050'), np.str_('6003051'), np.str_('6003090'), np.str_('6003091'), np.str_('6003130'), np.str_('6003131'), np.str_('6004050'), np.str_('6004051'), np.str_('6004090'), np.str_('6004091'), np.str_('600413

<ipython-input-7-5e7cde8d0c1e>:12: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(file_path, preload=True)


...

/usr/local/lib/python3.11/dist-packages/pymatreader/utils.py:168: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn(


Used Annotations descriptions: [np.str_('500105'), np.str_('500109'), np.str_('500113'), np.str_('500205'), np.str_('500209'), np.str_('500213'), np.str_('500305'), np.str_('500309'), np.str_('500313'), np.str_('500405'), np.str_('500409'), np.str_('500413'), np.str_('500505'), np.str_('500509'), np.str_('500513'), np.str_('500609'), np.str_('500613'), np.str_('500709'), np.str_('500713'), np.str_('500809'), np.str_('500813'), np.str_('500909'), np.str_('500913'), np.str_('501013'), np.str_('501113'), np.str_('501213'), np.str_('501313'), np.str_('6001050'), np.str_('6001051'), np.str_('6001090'), np.str_('6001091'), np.str_('6001130'), np.str_('6001131'), np.str_('6002050'), np.str_('6002051'), np.str_('6002090'), np.str_('6002091'), np.str_('6002130'), np.str_('6002131'), np.str_('6003051'), np.str_('6003090'), np.str_('6003091'), np.str_('6003130'), np.str_('6003131'), np.str_('6004050'), np.str_('6004051'), np.str_('6004090'), np.str_('6004091'), np.str_('6004130'), np.str_('600413

<ipython-input-7-5e7cde8d0c1e>:12: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(file_path, preload=True)


Not setting metadata
48 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 48 events and 21001 original time points ...
0 bad epochs dropped
Not setting metadata
48 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 48 events and 29001 original time points ...
0 bad epochs dropped


Processing subjects: 100%|██████████| 83/83 [1:26:33<00:00, 62.57s/it]


In [8]:
# Save as pickle
pickle_filename = os.path.join(output_root, "processed_ecg_data.pkl")
with open(pickle_filename, "wb") as f:
    pickle.dump({"subjects": subject_list, "data": final_data_array}, f)

print("\n ECG Processing Complete!")
print(f" Data shape: {final_data_array.shape}")
print(f" Subjects: {len(subject_list)} | Saved to: {pickle_filename}")


 ECG Processing Complete!
 Data shape: (65, 3, 3, 54, 26000)
 Subjects: 65 | Saved to: /content/output_ecg_data/processed_ecg_data.pkl
